In [1]:
import sys
sys.executable

'd:\\Code-Projects\\University\\Fall2023\\Grad\\LipNet-Assistive-AI\\.conda\\python.exe'

In [2]:
WORK_DIR = r"E:/Video Links Dataset"
CLIPPED_VIDEOS = WORK_DIR + r"/clipped/videos"
CLIPPED_ALIGNMENTS = WORK_DIR + r"/clipped/alignments"
DOWNLOADED_VIDEOS = WORK_DIR + r"/downloaded/videos"
DOWNLOADED_ALIGNMENTS = WORK_DIR + r"/downloaded/alignments"
FILTERED_VIDEOS = WORK_DIR + r"/filtered/videos"
FILTERED_ALIGNMENTS = WORK_DIR + r"/filtered/alignments"
LINK_DATASET = WORK_DIR + r"/Video Links Dataset.csv"
STATUS_DATASET = WORK_DIR + r"/Video Status.csv"
LOG_FILE = WORK_DIR + r"/log.txt"


In [3]:
from pytube import YouTube
from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import crop
from os import path, remove, makedirs, rename
from cv2 import VideoCapture,cvtColor, absdiff,COLOR_BGR2RGB,COLOR_BGR2GRAY,CAP_PROP_FPS,CAP_PROP_FRAME_COUNT
from mediapipe.python.solutions.face_mesh import FaceMesh
from mediapipe.python.solutions.face_detection import FaceDetection
from pandas import read_csv, concat,DataFrame
from numpy import mean
from time import sleep
import pysrt
import re 
from string import punctuation
from subprocess import run
from glob import glob
from shutil import copy
from matplotlib.pyplot import plot 


pygame 2.5.2 (SDL 2.28.3, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:

        




# def cut_from_srt_file(video_path, srt_path, output_dir_videos,output_dir_str, id):
    
#     subs = pysrt.open(srt_path)
#     video = VideoFileClip(video_path)

#     makedir(output_dir_videos, exist_ok=True)

#     for idx, sub in enumerate(subs):
#         try:
#             text = filter_srt_sub(sub.text.strip())

#             if( not text):
#                 continue

#             start_time = sub.start.ordinal /1000.0
#             end_time = sub.end.ordinal / 1000.0     

#             video_part = video.subclip(start_time, end_time)
#             output_path = path.join(output_dir_videos, f"{id}_{idx+1}.mp4")
            
#             with open(output_dir_str+f"/{id}_{idx+1}.align","w") as writer:
#                 writer.write(text.replace(" ","\n"))

#             video_part.write_videofile(output_path,verbose=False, progress_bar=False) 
#         except:
#             video.close()
#             pass
#     video.close()



# def filter_videos(videos_path,alignments_path):
#     for i in glob(videos_path+fr"/*.mp4"):
#         video_path = i.replace("\\","/")
#         alignment_path = alignments_path+"/"+re.split(r"\\|/",i)[-1].split(".")[0]+".align"

#         if(video_is_accpeted(video_path)):
#             copy(video_path, FILTERED_VIDEOS)
#             copy(alignment_path, FILTERED_ALIGNMENTS)




In [ ]:
def check_outliers(numbers,threshold):
    if not numbers:
        return False  # Return False if the list is empty

    mean = sum(numbers) / len(numbers)
    
    # Calculate standard deviation
    variance = sum((x - mean) ** 2 for x in numbers) / len(numbers)
    std_dev = variance ** 0.5
    
    # Define thresholds
    upper_threshold = mean + (threshold * std_dev)
    
    # Check if any number falls outside the thresholds
    for num in numbers:
        if num > upper_threshold:
            return True  # Return True if any number is outside the thresholds
    
    return False  # Return False if no number is outside the thresholds

def calculate_frame_differences(video_path):
    cap = VideoCapture(video_path)
    if not cap.isOpened():
        # print("Error: Couldn't open video file.")
        return None
    
    # Read the first frame
    ret, prev_frame = cap.read()
    if not ret:
        # print("Error: Couldn't read the first frame.")
        return None
    
    # Initialize list to store differences
    differences = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Calculate absolute difference between frames
        diff = absdiff(prev_frame, frame)
        
        # Convert difference image to grayscale
        gray_diff = cvtColor(diff, COLOR_BGR2GRAY)
        
        # Calculate the average pixel difference
        avg_diff = mean(gray_diff)
        differences.append(avg_diff)
        
        # Update the previous frame
        prev_frame = frame.copy()
    cap.release()
    # plt.plot(differences);
    return(check_outliers(differences,4))

def english_to_arabic_numbers(text):

    numbers_dict = {
        '0': '٠',
        '1': '١',
        '2': '٢',
        '3': '٣',
        '4': '٤',
        '5': '٥',
        '6': '٦',
        '7': '٧',
        '8': '٨',
        '9': '٩',
    }
    for eng_num, arabic_num in numbers_dict.items():
        text = text.replace(eng_num, arabic_num)
    return text

def remove_special_char(text):
    for i in punctuation:
        text = text.replace(i, " ")

    return text

def remove_repeating_spaces(text):
    cleaned_text = re.sub(r' +', ' ', text)
    return cleaned_text

def check_if_not_arabic(text):
    ar = "ابتثجحخدذرزسشصضطظعغفقكلمنهويىء٠١٢٣٤٥٦٧٨٩ "
    for i in text:
        if i not in ar:
            return True
    return False

def get_video(url, video_id):
    try:
        yt = YouTube(url)

        stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()

        if "a.ar" in yt.captions.keys():
            captions = yt.captions.get_by_language_code("a.ar")
            captions.download(
                srt=True,
                output_path=DOWNLOADED_ALIGNMENTS,
                title=f"{video_id}")

            stream.download(
                filename=fr"{DOWNLOADED_VIDEOS}/{video_id}.mp4"
            )

            return fr"{DOWNLOADED_VIDEOS}/{video_id}.mp4",fr"{DOWNLOADED_ALIGNMENTS}/{video_id} (a.ar).srt"

        else:
            return False, False
        
    except Exception as e:
        return None,None
    
def video_is_accpeted(video_path):

    try:
        # open video file
        cap = VideoCapture(video_path)
        if not cap.isOpened():
            raise Exception("can't open video file")
        
        # setup models
        face_mesh = FaceMesh(min_detection_confidence=0.8)
        face_detection = FaceDetection(model_selection=2, min_detection_confidence=0.8)

        video_fps = int(cap.get(CAP_PROP_FPS))
        video_frames = int(cap.get(CAP_PROP_FRAME_COUNT))
        video_duration = video_frames // video_fps

        if (int(video_duration) < 2) or (int(video_duration) > 5):
            raise Exception("video_duration must be between 2 and 5")
        
        if((video_fps < 25) or (video_fps > 60)):
            raise Exception("video fps must be between 25 and 60")

        # lip coordinates,
        lip_pairs = [[13, 14], [82, 87], [312, 317],[81,178],[311,402],[310,318],[80,88]]
        # list for whether face is facing us
        orientation_list=[]
        # list for whether frames are lips are moving rapidly
        lip_diffs = []

        # loop over all frames in the video
        for _ in range(int(cap.get(CAP_PROP_FRAME_COUNT))):
            ret, frame = cap.read()

            if not ret:
                raise Exception("couldn't read frame")
            
            # convert to rgb
            frame = cvtColor(frame, COLOR_BGR2RGB)

            # get the keypoints (x,y)s of each point of a face in the frame
            results = face_detection.process(frame)

            if not results.detections:
                raise Exception("no faces detected")
            
            if(len(results.detections) !=1):
                raise Exception("wrong number of faces detected")


            face_mesh_result = face_mesh.process(frame)
            if not face_mesh_result.multi_face_landmarks:
                raise Exception("no lips detected")
            
            for face_landmarks in face_mesh_result.multi_face_landmarks:

                landmarks = face_landmarks.landmark
                right_x = int(landmarks[454].x * frame.shape[1])
                left_x = int(landmarks[234].x * frame.shape[1])
                upper_x = int(landmarks[10].x * frame.shape[1])
                lower_x = int(landmarks[152].x * frame.shape[1])

                center_x = (upper_x + lower_x)//2

                if(left_x>center_x):
                    orientation_list.append(0)
                elif(right_x<center_x):
                    orientation_list.append(0)
                else:
                    orientation_list.append(1)
                    
                diff = 0
                for lip_pair in lip_pairs:
                    upper_lip_x, upper_lip_y = int(
                        landmarks[lip_pair[0]].x * frame.shape[1]
                    ), int(landmarks[lip_pair[0]].y * frame.shape[0])
                    lower_lip_x, lower_lip_y = int(
                        landmarks[lip_pair[1]].x * frame.shape[1]
                    ), int(landmarks[lip_pair[1]].y * frame.shape[0])
                    diff += ((upper_lip_x - lower_lip_x) ** 2) + (
                        (upper_lip_y - lower_lip_y) ** 2
                    )
                lip_diffs.append(diff)
            
        if(len(orientation_list) == 0):
            raise Exception("Face border coudn't detect any face for orientation")
        if(len(lip_diffs) == 0 ):
            raise Exception("Face mesh couldn't detect any lips")
        
        if((sum(orientation_list)/len(orientation_list)) !=1.0):
            raise Exception("too much face angle")

        if(not check_outliers(lip_diffs,1)):
            raise Exception("lips not moving")

        cap.release()
        
        if calculate_frame_differences(video_path):
            raise Exception("sudden change in frames")
            
    except Exception as e:
        raise e
    

### logging and control functions ###

def empty_dir(dir):
    for i in glob(dir+r"/*"):
        remove(i)
        
def log(text):
    try:
        with open(LOG_FILE, 'a') as file:
            file.write(text)
    except Exception as e:
        print("Failed to log: ",text, e)
def save_pipeline_status(video_idx,status):
    df = read_csv(LINK_DATASET)
    df["status"][video_idx] = status
    df.to_csv(LINK_DATASET,index=False)    

def log_status_video(video_id, status):
    status_df = read_csv(STATUS_DATASET)
    new_row = {'video_id': video_id, 'status': status}
    status_df = concat([status_df, DataFrame([new_row])], ignore_index=True)
    status_df.to_csv(STATUS_DATASET,index=False)    


In [ ]:
def download_video(url,video_id):
    log(f"Downloading video: {video_id}\n\n")

    try:
                
        fails = 10

        output_video, output_alignment = get_video(url,video_id)
        while (output_video == None) and (fails > 0):
            fails -= 1
            sleep(4)
            output_video, output_alignment = download_video(url,video_id)

        if not output_video:
            raise Exception(f"Video not downloading after {fails} times")

        # fix srt file
        run(["go","run",".",output_alignment])
        remove(fr"{DOWNLOADED_ALIGNMENTS}/{video_id} (a.ar).srt")
        rename(fr"{DOWNLOADED_ALIGNMENTS}/{video_id} (a.ar).srt.fixed",fr"{DOWNLOADED_ALIGNMENTS}/{video_id}.srt")
        log(f"Finished Downloading video: {video_id}\n\n")

    except Exception as e:
        raise e
     
def clip_video(video_path, alignment_path,video_id):
    log(f"Clipping videos: {video_path}\n\n")

    def filter_srt_sub(text):
    
        if(len(text) < 2):
            return False

        if(text == "[موسيقى]"):
            return False
        
        text = english_to_arabic_numbers(text)
        text = remove_special_char(text)
        text = remove_repeating_spaces(text)
        
        if(check_if_not_arabic(text)):
            return False
        
        if(len(text) < 2):
            return False
        
        return text
    try:
        subs = pysrt.open(alignment_path)
        video = VideoFileClip(video_path)

        for idx, sub in enumerate(subs):
            try:
                text = filter_srt_sub(sub.text.strip())

                if(not text):
                    continue

                start_time = sub.start.ordinal /1000.0
                end_time = sub.end.ordinal / 1000.0     

                video_part = video.subclip(start_time, end_time)
                output_path = path.join(CLIPPED_VIDEOS, f"{video_id}_{idx+1}.mp4")
                
                with open(CLIPPED_ALIGNMENTS+f"/{video_id}_{idx+1}.align","w") as writer:
                    writer.write(text.replace(" ","\n"))

                video_part.write_videofile(output_path,verbose=False, progress_bar=False) 
            except:
                video.close()
        video.close()
        log(f"Finished Clipping videos: {video_path}\n\n")

    except:
        raise Exception("Failed to open srt file or video file")

def filter_video():
    log("Filtering Videos\n\n")
    try:
        for i in glob(CLIPPED_VIDEOS+fr"/*.mp4"):
            video_id = re.split(r"\\|/",i)[-1].split(".")[0]
            video_path = i.replace("\\","/")
            alignment_path = CLIPPED_ALIGNMENTS+"/"+video_id+".align"


            try:
                video_is_accpeted(video_path)
                copy(video_path,FILTERED_VIDEOS)
                copy(alignment_path,FILTERED_ALIGNMENTS)
                log_status_video(video_id,"parsed")
            except Exception as e:
                log_status_video(video_id,e)
        log("\nFinished Filtering Videos\n\n")
    except Exception as e:
        raise e

In [ ]:
def pipline(url, video_id,status,video_idx):
    log(f"{'='*100}\n\n")
    log(f"Start Pipline for id: {video_id} with status: {status}\n\n")
    try:
        if status == "parsed":
            log(f"Finished Pipline for id: {video_id}\n\n")
            log(f"{'='*100}\n\n")
            return "parsed"
        
        elif status == "downloaded":
            empty_dir(CLIPPED_VIDEOS)
            empty_dir(CLIPPED_ALIGNMENTS)

            clip_video(DOWNLOADED_VIDEOS+fr"/{video_id}.mp4", DOWNLOADED_ALIGNMENTS+fr"/{video_id}.srt",video_id)
            save_pipeline_status(video_idx,"clipped")

            filter_video()
            empty_dir(CLIPPED_VIDEOS)
            empty_dir(CLIPPED_ALIGNMENTS)
            empty_dir(DOWNLOADED_VIDEOS)
            empty_dir(DOWNLOADED_ALIGNMENTS)
            log(f"Finished Pipline for id: {video_id}\n\n")
            log(f"{'='*100}\n\n")
            return "parsed"

        elif status == "clipped":
            filter_video()
            empty_dir(CLIPPED_VIDEOS)
            empty_dir(CLIPPED_ALIGNMENTS)
            empty_dir(DOWNLOADED_VIDEOS)
            empty_dir(DOWNLOADED_ALIGNMENTS)
            log(f"Finished Pipline for id: {video_id}\n\n")
            log(f"{'='*100}\n\n")
            return "parsed"

            
        elif status == "failed" or status == "unparsed":
            empty_dir(CLIPPED_VIDEOS)
            empty_dir(CLIPPED_ALIGNMENTS)
            empty_dir(DOWNLOADED_VIDEOS)
            empty_dir(DOWNLOADED_ALIGNMENTS)

            download_video(url,video_id)
            save_pipeline_status(video_idx,"downloaded")

            clip_video(DOWNLOADED_VIDEOS+fr"/{video_id}.mp4", DOWNLOADED_ALIGNMENTS+fr"/{video_id}.srt",video_id)
            save_pipeline_status(video_idx,"clipped")

            filter_video()

            empty_dir(CLIPPED_VIDEOS)
            empty_dir(CLIPPED_ALIGNMENTS)
            empty_dir(DOWNLOADED_VIDEOS)
            empty_dir(DOWNLOADED_ALIGNMENTS)
            log(f"Finished Pipline for id: {video_id}\n\n")
            log(f"{'='*100}\n\n")
            return "parsed"
        
    except Exception as e:
        log(f"ERROR Pipeline for {video_id}:\n\n")
        log(f"Finished Pipline for id: {video_id}\n\n")
        log(f"{'='*100}\n\n")
        log(f"{e}\n\n")
        empty_dir(CLIPPED_VIDEOS)
        empty_dir(CLIPPED_ALIGNMENTS)
        empty_dir(DOWNLOADED_VIDEOS)
        empty_dir(DOWNLOADED_ALIGNMENTS)
        return e    
    
    

In [8]:
df = read_csv(LINK_DATASET)
df.head()

,link,name,id,creator,status
0,https://www.youtube.com/watch?v=s0pbzSC81Kc,كتابة الكلام على الفيديو بشكل تلقائي بالذكاء ا...,0,0,unparsed
1,https://www.youtube.com/watch?v=FYqNG3zwXL4&t=...,فيلم الصفاره,1,0,unparsed
2,https://www.youtube.com/watch?v=3r3IOzNNKRM,جت كدا اياد الموجي وساره هاني,2,0,unparsed
3,https://www.youtube.com/watch?v=Z31F0zVuHOg&t=...,فيلم العارف,3,0,unparsed
4,https://www.youtube.com/watch?v=0SGpG7m4xzE,فيلم الخليه,4,0,unparsed


In [9]:
df[df.duplicated('link',keep=False)]

,link,name,id,creator,status


In [10]:
for idx,row in enumerate(df.values):
    url = row[0]
    video_id = row[2]
    status = row[4]
    break
    # run pipline
    result = pipline(url, video_id,status,idx)
    # # save completion
    save_pipeline_status(idx,result)
    # df["status"][idx] = result
    # df.to_csv(LINK_DATASET,index=False)    
    print("Parsed Video : ",video_id)


In [11]:
result = pipline(url, video_id,status)


In [ ]:
# code in captions.py
    # def xml_caption_to_srt(self, xml_captions: str) -> str:
    #     """Convert xml caption tracks to "SubRip Subtitle (srt)".

    #     :param str xml_captions:
    #     XML formatted caption tracks.
    #     """
    #     segments = []
    #     root = ElementTree.fromstring(xml_captions)
    #     i=0
    #     for child in list(root.iter("body"))[0]:
    #         if child.tag == 'p':
    #             caption = ''
    #             if len(list(child))==0:
    #                 # instead of 'continue'
    #                 caption = child.text
    #             for s in list(child):
    #                 if s.tag == 's':
    #                     caption += ' ' + s.text
    #             caption = unescape(caption.replace("\n", " ").replace("  ", " "),)
    #             try:
    #                 duration = float(child.attrib["d"])/1000.0
    #             except KeyError:
    #                 duration = 0.0
    #             start = float(child.attrib["t"])/1000.0
    #             end = start + duration
    #             sequence_number = i + 1  # convert from 0-indexed to 1.
    #             line = "{seq}\n{start} --> {end}\n{text}\n".format(
    #                 seq=sequence_number,
    #                 start=self.float_to_srt_time_format(start),
    #                 end=self.float_to_srt_time_format(end),
    #                 text=caption,
    #             )
    #             segments.append(line)
    #             i += 1
    #     return "\n".join(segments).strip()